In [1]:
import os
import pandas as pd
import pathlib
import sqlalchemy as sa
import highered as h
import zipfile
pd.set_option('display.max_colwidth', 100)

In [14]:
db = sa.create_engine('sqlite:///ignoredir/highered.sqlite')
zipname = r'ignoredir/CollegeScorecard_Raw_Data_08032021.zip'
specialnull = 'PrivacySuppressed'
p = pathlib.Path('./ignoredir')
p

WindowsPath('ignoredir')

In [3]:
for f in p.iterdir():
    print(f)

ignoredir\CDS_2021-2022-1.xlsx
ignoredir\CollegeScorecard_Raw_Data_08032021.zip
ignoredir\EdStatsCountry-Series.csv
ignoredir\EdStatsData.csv
ignoredir\EdStatsFootNote.csv
ignoredir\EdStatsSeries.csv
ignoredir\highered.sqlite
ignoredir\ipeds_gr200_20.csv
ignoredir\ipeds_gr2020.csv
ignoredir\ipeds_gr2020_l2.csv
ignoredir\ipeds_gr2020_pell_ssl.csv
ignoredir\kag_dps_Salary Dataset(bangalore).csv
ignoredir\kag_usdoe_database.sqlite
ignoredir\kag_usdoe_data_dictionary.yaml
ignoredir\kag_worldbank_EdStatsCountry.csv
ignoredir\kag_wsj_degrees-that-pay-back.csv
ignoredir\kag_wsj_salaries-by-college-type.csv
ignoredir\kag_wsj_salaries-by-region.csv
ignoredir\NCES_2020239_1.1-A_Estimates.xlsx
ignoredir\nces_user_BB0812_PostBAenrollment.xlsx


# list of all files in the zip

In [5]:
[f.filename for f in z.filelist]

['Crosswalks.zip',
 '__MACOSX/._Crosswalks.zip',
 'data.yaml',
 'FieldOfStudyData1415_1516_PP.csv',
 'FieldOfStudyData1516_1617_PP.csv',
 'FieldOfStudyData1617_1718_PP.csv',
 'MERGED1996_97_PP.csv',
 'MERGED1997_98_PP.csv',
 'MERGED1998_99_PP.csv',
 'MERGED1999_00_PP.csv',
 'MERGED2000_01_PP.csv',
 'MERGED2001_02_PP.csv',
 'MERGED2002_03_PP.csv',
 'MERGED2003_04_PP.csv',
 'MERGED2004_05_PP.csv',
 'MERGED2005_06_PP.csv',
 'MERGED2006_07_PP.csv',
 'MERGED2007_08_PP.csv',
 'MERGED2008_09_PP.csv',
 'MERGED2009_10_PP.csv',
 'MERGED2010_11_PP.csv',
 'MERGED2011_12_PP.csv',
 'MERGED2012_13_PP.csv',
 'MERGED2013_14_PP.csv',
 'MERGED2014_15_PP.csv',
 'MERGED2015_16_PP.csv',
 'MERGED2016_17_PP.csv',
 'MERGED2017_18_PP.csv',
 'MERGED2018_19_PP.csv',
 'MERGED2019_20_PP.csv',
 'Most-Recent-Cohorts-All-Data-Elements.csv',
 'Most-Recent-Cohorts-Field-of-Study.csv']

In [74]:
alldata_files = [x.filename for x in z.filelist if 'MERGED' in x.filename or 'all-data' in x.filename.lower()]
FOS_files = [x.filename for x in z.filelist if 'field' in x.filename.lower()]

# test loading zipfiles using module functions

In [79]:
df = h.df_from_zipcsv(zipname, 'MERGED1998_99_PP.csv')

In [80]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6702 entries, 0 to 6701
Columns: 2393 entries, UNITID to filename
dtypes: float64(2365), int64(11), object(17)
memory usage: 122.4+ MB


## load all merged (all data elements) files

In [81]:
%%time
bigdf = pd.DataFrame()
for f in alldata_files:
    print(f, end="|")
    df = h.df_from_zipcsv(zipname, f)
    df['filename'] = f
    bigdf = bigdf.append(df)
bigdf.info()

MERGED1996_97_PP.csv|MERGED1997_98_PP.csv|MERGED1998_99_PP.csv|MERGED1999_00_PP.csv|MERGED2000_01_PP.csv|MERGED2001_02_PP.csv|MERGED2002_03_PP.csv|MERGED2003_04_PP.csv|MERGED2004_05_PP.csv|MERGED2005_06_PP.csv|MERGED2006_07_PP.csv|MERGED2007_08_PP.csv|MERGED2008_09_PP.csv|MERGED2009_10_PP.csv|MERGED2010_11_PP.csv|MERGED2011_12_PP.csv|MERGED2012_13_PP.csv|MERGED2013_14_PP.csv|MERGED2014_15_PP.csv|MERGED2015_16_PP.csv|MERGED2016_17_PP.csv|MERGED2017_18_PP.csv|MERGED2018_19_PP.csv|MERGED2019_20_PP.csv|Most-Recent-Cohorts-All-Data-Elements.csv|<class 'pandas.core.frame.DataFrame'>
Int64Index: 176720 entries, 0 to 6693
Columns: 2393 entries, UNITID to filename
dtypes: float64(2366), int64(6), object(21)
memory usage: 3.2+ GB
Wall time: 2min 14s


In [83]:
bigdf.describe()

,UNITID,OPEID6,SCH_DEG,HCM2,MAIN,NUMBRANCH,PREDDEG,HIGHDEG,CONTROL,ST_FIPS,...,LPGPLUS_AMT,BOOKSUPPLY,ROOMBOARD_ON,OTHEREXPENSE_ON,ROOMBOARD_OFF,OTHEREXPENSE_OFF,OTHEREXPENSE_FAM,ENDOWBEGIN,ENDOWEND,DOLPROVIDER
count,1.767200e+05,176720.000000,145337.000000,13388.000000,176720.000000,176720.000000,176720.000000,176720.000000,176691.000000,176719.000000,...,1.276900e+04,85717.000000,43797.000000,43568.000000,80442.000000,80304.000000,80599.000000,4.916700e+04,4.916700e+04,13388.000000
mean,1.043344e+06,14734.849785,1.951196,0.009262,0.796961,4.021752,1.875990,2.206355,2.118569,29.204245,...,2.236023e+07,1088.776719,7667.395735,2686.415328,7723.227058,3486.726875,3661.378206,1.491571e+08,1.579482e+08,0.384374
std,4.913349e+06,12721.426496,0.880057,0.095796,0.402262,13.101871,1.005295,1.292524,0.837590,16.606854,...,1.016471e+08,625.285476,3146.819262,1400.639991,3228.433649,1880.026710,2019.317542,1.035883e+09,1.090506e+09,0.486465
min,1.006360e+05,1002.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,...,-1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000
25%,1.641550e+05,3242.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,13.000000,...,-1.000000e+00,760.000000,5355.000000,1700.000000,5607.000000,2200.000000,2306.000000,1.999304e+06,2.169231e+06,0.000000
50%,2.136175e+05,9859.500000,2.000000,0.000000,1.000000,1.000000,2.000000,2.000000,2.000000,29.000000,...,-1.000000e+00,1000.000000,7310.000000,2500.000000,7272.000000,3240.000000,3340.000000,1.042499e+07,1.110630e+07,0.000000
75%,4.136350e+05,23522.000000,3.000000,0.000000,1.000000,2.000000,3.000000,4.000000,3.000000,42.000000,...,4.893060e+05,1350.000000,9636.000000,3410.000000,9488.000000,4422.000000,4585.000000,4.548325e+07,4.809454e+07,1.000000
max,4.917560e+07,99999.000000,3.000000,1.000000,1.000000,143.000000,4.000000,4.000000,3.000000,78.000000,...,2.608959e+09,28000.000000,25668.000000,32600.000000,106962.000000,33081.000000,50000.000000,3.923374e+10,4.092970e+10,1.000000


In [6]:
h.get_csv_from_zipfile(zipname, 'MERGED2014_15_PP.csv')  

C:\Users\Rob.DESKTOP-HBG5EOT\.conda\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (6,1537,1540,1542,1575,1589,1606,1610,1614,1615,1729,1909,1910,1911,1912,1913) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


,UNITID,OPEID,OPEID6,INSTNM,CITY,STABBR,ZIP,ACCREDAGENCY,INSTURL,NPCURL,...,FEDSCHCD,BOOKSUPPLY,ROOMBOARD_ON,OTHEREXPENSE_ON,ROOMBOARD_OFF,OTHEREXPENSE_OFF,OTHEREXPENSE_FAM,ENDOWBEGIN,ENDOWEND,DOLPROVIDER
0,100654,100200,1002,Alabama A & M University,Normal,AL,35762,NaN,NaN,NaN,...,NaN,1800.0,7240.0,2748.0,10119.0,2748.0,1300.0,NaN,NaN,NaN
1,100663,105200,1052,University of Alabama at Birmingham,Birmingham,AL,35294-0110,NaN,NaN,NaN,...,NaN,1200.0,10809.0,4886.0,10809.0,4886.0,4886.0,392284979.0,424547661.0,NaN
2,100690,2503400,25034,Amridge University,Montgomery,AL,36117-3553,NaN,NaN,NaN,...,NaN,1500.0,NaN,NaN,9600.0,1600.0,1600.0,174805.0,174804.0,NaN
3,100706,105500,1055,University of Alabama in Huntsville,Huntsville,AL,35899,NaN,NaN,NaN,...,NaN,1639.0,8433.0,4302.0,8433.0,4302.0,4302.0,69929806.0,74366851.0,NaN
4,100724,100500,1005,Alabama State University,Montgomery,AL,36104-0271,NaN,NaN,NaN,...,NaN,1600.0,5422.0,2544.0,7320.0,3172.0,3172.0,73546541.0,86468050.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7761,48065701,869423,8694,Rasmussen College - Overland Park,Overland Park,KS,662102786,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7762,48154401,4220901,42209,National Personal Training Institute of Cleveland,Highland Heights,OH,44143,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7763,48387801,4223701,42237,Bay Area Medical Academy - San Jose Satellite Location,San Jose,CA,95113,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7764,48400201,4228101,42281,High Desert Medical College - Bakerfield,Bakersfield,CA,93301,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# load data dictionary from yaml file

In [35]:
h.read_file_from_zip(zipname, 'data.yaml')

b'---\nversion: 2021-07-19-09:50-0700\napi: schools\nindex: school-data\nunique:\n  - id\noptions:\n  columns: all\n  search: dictionary_only\nnull_value:\n  - \'NULL\'\n  - PrivacySuppressed\nexamples:\n  - name: College Scorecard Search results\n    description: the default scorecard universe\n    params: \'/v1/schools?page=0&sort=latest.completion.consumer_rate:desc&school.operating=1&latest.student.size__range=1..&latest.academics.program_available.assoc_or_bachelors_or_certificate=true&school.degrees_awarded.predominant__range=1..3&fields=id,school.name,school.city,school.state,latest.student.size,school.branches,school.locale,school.ownership,school.degrees_awarded.predominant,latest.academics.program_reporter.programs_offered,latest.cost.avg_net_price.overall,latest.completion.consumer_rate,latest.earnings.10_yrs_after_entry.median,latest.earnings.6_yrs_after_entry.percent_greater_than_25000,school.under_investigation,latest.completion.outcome_percentage_suppressed.all_students.

In [14]:
f = z.open('data.yaml')
s = f.read().decode()

## test loading yaml using pyyaml
this works for files and examples but not for the dictionary section

In [17]:
import yaml
ydd = yaml.safe_load(z.open('data.yaml'))

In [18]:
ydd.keys()

dict_keys(['version', 'api', 'index', 'unique', 'options', 'null_value', 'examples', 'dictionary', 'files'])

In [26]:
pd.DataFrame(ydd['files'])

,name,key,map
0,Most-Recent-Cohorts-All-Data-Elements.csv,latest,NaN
1,Most-Recent-Cohorts-Field-of-Study.csv,latest,program_data
2,MERGED2019_20_PP.csv,2019,NaN
3,MERGED2018_19_PP.csv,2018,NaN
4,MERGED2017_18_PP.csv,2017,NaN
5,FieldOfStudyData1617_1718_PP.csv,2017,program_data
6,MERGED2016_17_PP.csv,2016,NaN
7,FieldOfStudyData1516_1617_PP.csv,2016,program_data
8,MERGED2015_16_PP.csv,2015,NaN
9,FieldOfStudyData1415_1516_PP.csv,2015,program_data


## manual parsing of yaml file
this gives us more control over the multiple levels of nesting in the dictionary  section

In [59]:
linelist = s.splitlines()#[x.split(":") for x in s.splitlines()]

In [76]:
data_dict = {}
key = ''
for l in linelist:
    if l[0] != ' ':
        keyval = l.split(':')
        if len(keyval) > 1:
            key = keyval[0].strip()
            data_dict[key] = [keyval[1].strip(),]
        else:
            key = l.strip()
            data_dict[key] = []
    else:
        data_dict[key].append(l.split(':'))

In [79]:
data_dict.keys()

dict_keys(['---', 'version', 'api', 'index', 'unique', 'options', 'null_value', 'examples', 'dictionary', 'files'])

### split yaml sections

In [97]:
examples = data_dict['examples']
dictionary = data_dict['dictionary']
files = data_dict['files']

In [162]:
#dictionary is more complex than other fields with multiple levels of nesting
lines = dictionary
keyfield = 'id'
fl = []
curdict = {}
i = -1
while i < len(lines)-1:
    i+=1
    #print(f"i is {i}")
    f = lines[i]
    if len(f)<1: continue
    if f[0][3] != ' ': 
        #flush the old dict
        if len(curdict) > 0: 
            fl.append(curdict)
        #start a new dict
        curdict = {}     
        curdict[keyfield] = f[0]
    elif 'description' in f[0] and '|-' in f[1]:
        i=i+1

        print('multi-line description', f, lines[i])
        curdict['description'] = lines[i][0].strip()
        while True:
            nextline = lines[i+1][0].strip() #remove leading whitespace
            if nextline[0].isdigit():
                print(nextline)
                curdict['lvl_'+nextline[0]] = nextline[2:]
                i+=1
                #print(f"i{i}")
            else:
                #print(f"j is {j}")
                break
        #i=j+1
        
    else:
        try:
            curdict[f[0].strip()] = f[1].strip()
        except IndexError as e:
            print(e)
            print(f)

datadf = pd.DataFrame(fl)


multi-line description ['    description', ' |-'] ['      Predominant undergraduate degree awarded']
0 Not classified
1 Predominantly certificate-degree granting
2 Predominantly associate's-degree granting
3 Predominantly bachelor's-degree granting
4 Entirely graduate-degree granting
multi-line description ['    description', ' |-'] ['      Highest degree awarded']
0 Non-degree-granting
1 Certificate degree
2 Associate degree
3 Bachelor's degree
4 Graduate degree


In [ ]:
datadf.to_sql('dictionary', db, if_exists='replace', index=False)

In [46]:
datadf = pd.read_sql('select * from dictionary', db)

In [168]:
datadf[datadf['map'].notnull()]

,id,source,type,description,index,lvl_0,lvl_1,lvl_2,lvl_3,lvl_4,map,calculate
2392,programs.cip_4_digit.unit_id,P_UNITID,integer,Unit ID for institution,NaN,NaN,NaN,NaN,NaN,NaN,program,NaN
2393,programs.cip_4_digit.ope6_id,P_OPEID6,NaN,6-digit OPE ID for institution,NaN,NaN,NaN,NaN,NaN,NaN,program,NaN
2394,programs.cip_4_digit.school.name,P_INSTNM,autocomplete,Institution name,NaN,NaN,NaN,NaN,NaN,NaN,program,NaN
2395,programs.cip_4_digit.school.type,P_CONTROL,integer,Control of institution,NaN,NaN,NaN,NaN,NaN,NaN,program,NaN
2396,programs.cip_4_digit.school.main_campus,P_MAIN,integer,Flag for main campus,NaN,NaN,NaN,NaN,NaN,NaN,program,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2479,programs.cip_4_digit.repayment.2_yr_bb_fed_comp.deferment,P_BBRR2_FED_COMP_DFR,float,Percentage of undergraduate completer undergraduate federal student loan borrowers in deferment ...,NaN,NaN,NaN,NaN,NaN,NaN,program,NaN
2480,programs.cip_4_digit.repayment.2_yr_bb_fed_comp.noprogress,P_BBRR2_FED_COMP_NOPROG,float,Percentage of undergraduate completer undergraduate federal student loan borrowers not making pr...,NaN,NaN,NaN,NaN,NaN,NaN,program,NaN
2481,programs.cip_4_digit.repayment.2_yr_bb_fed_comp.makingprogress,P_BBRR2_FED_COMP_MAKEPROG,float,Percentage of undergraduate completer undergraduate federal student loan borrowers making progre...,NaN,NaN,NaN,NaN,NaN,NaN,program,NaN
2482,programs.cip_4_digit.repayment.2_yr_bb_fed_comp.fullypaid,P_BBRR2_FED_COMP_PAIDINFULL,float,Percentage of undergraduate completer undergraduate federal student loan borrowers paid in full ...,NaN,NaN,NaN,NaN,NaN,NaN,program,NaN


count    1.767200e+05
mean     1.043344e+06
std      4.913349e+06
min      1.006360e+05
25%      1.641550e+05
50%      2.136175e+05
75%      4.136350e+05
max      4.917560e+07
Name: UNITID, dtype: float64

In [94]:
2386/2

1193.0

In [ ]:
indexcols = ['UNITID', 'OPEID', 'OPEID6', 'INSTNM', 'CITY', 'STABBR', 'ZIP']
nonindexcols = alldatadf.columns[~alldatadf.columns.isin(indexcols)]

In [99]:
%%time
alldatadf[indexcols+list(nonindexcols[:1193])].to_sql('all_data_fields1', db, index=False, if_exists='replace')

Wall time: 10min 7s


In [100]:
%%time
alldatadf[indexcols+list(nonindexcols[1193:])].to_sql('all_data_fields2', db, index=False, if_exists='replace')

Wall time: 8min 28s


In [44]:
#works for examples, files but not dictionary
i = sa.inspect(db)
i.get_table_names()
if 'files' in i.get_table_names():
    print('found files in db')
    filedf = pd.read_sql('select * from files', db)
else:
    print('files not found in db')
    filedf = h.yaml_to_df(files,)
    filedf.to_sql('files', db, index=False, if_exists='replace')

exampledf.sample(10)

found files in db


,name,description,params
2,"Two and four year schools, sorted by cost","Using the ""predominant degree"" at a school, showing name & cost, sorted by cost","'/v1/schools?school.degrees_awarded.predominant__range=2..3&_fields=school.name,id,latest.cost.a..."
0,College Scorecard Search results,the default scorecard universe,'/v1/schools?page=0&sort=latest.completion.consumer_rate
6,median earnings,"median earnings, sort descending (include name, id)","'/v1/schools?_fields=school.name,id,latest.earnings.10_yrs_after_entry.median&_sort=latest.earni..."
4,SF Bay Schools,"within 100mi of SF (include name, id)","'/v1/schools?_zip=91423&_distance=100mi&_fields=school.name,id,latest.earnings.6_yrs_after_entry..."
7,average net price,display school name and cost by average net price descending,"'/v1/schools?_fields=latest.cost.avg_net_price.overall,school.name,latest.student.size&_sort=lat..."
3,Four Year School with over 2000 students,"compare admission rate and completion rate, showing size and school identifying information","'/v1/schools?school.degrees_awarded.predominant=3&latest.student.size__range=2000..&_fields=id,l..."
9,Arizona State University,show school data for all years by id,'/v1/schools?id=104151&all_years=true'
8,Miami Dade College,show school data for all years by id,'/v1/schools?id=135717&all_years=true'
5,lowest tercile earnings,"mean earnings, lowest tercile, sort descending (include name, id)","'/v1/schools?_fields=school.name,id,latest.earnings.6_yrs_after_entry.mean_earnings.lowest_terci..."
1,Completion Rate,"schools with a graduation rate of 95% and above, sorted descending, displaying name and student ...",'/v1/schools?latest.completion.rate_suppressed.overall__range=0.95..&_fields=latest.completion.r...


In [45]:
if 'examples' in i.get_table_names():
    print('found examples in db')
    exampledf = pd.read_sql('select * from examples', db)
else:
    print('examples not found in db')
    exampledf = h.yaml_to_df(examples)
    exampledf.to_sql('examples', db, index=False, if_exists='replace')

exampledf.sample(10)

found examples in db


,name,description,params
2,"Two and four year schools, sorted by cost","Using the ""predominant degree"" at a school, showing name & cost, sorted by cost","'/v1/schools?school.degrees_awarded.predominant__range=2..3&_fields=school.name,id,latest.cost.a..."
6,median earnings,"median earnings, sort descending (include name, id)","'/v1/schools?_fields=school.name,id,latest.earnings.10_yrs_after_entry.median&_sort=latest.earni..."
1,Completion Rate,"schools with a graduation rate of 95% and above, sorted descending, displaying name and student ...",'/v1/schools?latest.completion.rate_suppressed.overall__range=0.95..&_fields=latest.completion.r...
4,SF Bay Schools,"within 100mi of SF (include name, id)","'/v1/schools?_zip=91423&_distance=100mi&_fields=school.name,id,latest.earnings.6_yrs_after_entry..."
7,average net price,display school name and cost by average net price descending,"'/v1/schools?_fields=latest.cost.avg_net_price.overall,school.name,latest.student.size&_sort=lat..."
0,College Scorecard Search results,the default scorecard universe,'/v1/schools?page=0&sort=latest.completion.consumer_rate
9,Arizona State University,show school data for all years by id,'/v1/schools?id=104151&all_years=true'
8,Miami Dade College,show school data for all years by id,'/v1/schools?id=135717&all_years=true'
5,lowest tercile earnings,"mean earnings, lowest tercile, sort descending (include name, id)","'/v1/schools?_fields=school.name,id,latest.earnings.6_yrs_after_entry.mean_earnings.lowest_terci..."
3,Four Year School with over 2000 students,"compare admission rate and completion rate, showing size and school identifying information","'/v1/schools?school.degrees_awarded.predominant=3&latest.student.size__range=2000..&_fields=id,l..."


,name,description,params
0,College Scorecard Search results,the default scorecard universe,'/v1/schools?page=0&sort=latest.completion.consumer_rate
1,Completion Rate,"schools with a graduation rate of 95% and above, sorted descending, displaying name and student ...",'/v1/schools?latest.completion.rate_suppressed.overall__range=0.95..&_fields=latest.completion.r...
2,"Two and four year schools, sorted by cost","Using the ""predominant degree"" at a school, showing name & cost, sorted by cost","'/v1/schools?school.degrees_awarded.predominant__range=2..3&_fields=school.name,id,latest.cost.a..."
3,Four Year School with over 2000 students,"compare admission rate and completion rate, showing size and school identifying information","'/v1/schools?school.degrees_awarded.predominant=3&latest.student.size__range=2000..&_fields=id,l..."
4,SF Bay Schools,"within 100mi of SF (include name, id)","'/v1/schools?_zip=91423&_distance=100mi&_fields=school.name,id,latest.earnings.6_yrs_after_entry..."
5,lowest tercile earnings,"mean earnings, lowest tercile, sort descending (include name, id)","'/v1/schools?_fields=school.name,id,latest.earnings.6_yrs_after_entry.mean_earnings.lowest_terci..."
6,median earnings,"median earnings, sort descending (include name, id)","'/v1/schools?_fields=school.name,id,latest.earnings.10_yrs_after_entry.median&_sort=latest.earni..."
7,average net price,display school name and cost by average net price descending,"'/v1/schools?_fields=latest.cost.avg_net_price.overall,school.name,latest.student.size&_sort=lat..."
8,Miami Dade College,show school data for all years by id,'/v1/schools?id=135717&all_years=true'
9,Arizona State University,show school data for all years by id,'/v1/schools?id=104151&all_years=true'


In [182]:
filedf.sample(10)

,name,key,map
18,MERGED2006_07_PP.csv,2006,NaN
2,MERGED2019_20_PP.csv,2019,NaN
0,Most-Recent-Cohorts-All-Data-Elements.csv,latest,NaN
7,FieldOfStudyData1516_1617_PP.csv,2016,program_data
3,MERGED2018_19_PP.csv,2018,NaN
8,MERGED2015_16_PP.csv,2015,NaN
23,MERGED2001_02_PP.csv,2001,NaN
16,MERGED2008_09_PP.csv,2008,NaN
25,MERGED1999_00_PP.csv,1999,NaN
15,MERGED2009_10_PP.csv,2009,NaN


In [85]:
%%time
alldatadf = pd.DataFrame() #pd.read_csv(z.open(recentf))
for x in alldata_files:
    print(x)
    df = pd.read_csv(z.open(x))
    df['filename'] = x
    alldatadf = alldatadf.append(df)
alldatadf.info()

MERGED1996_97_PP.csv


C:\Users\Rob.DESKTOP-HBG5EOT\.conda\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


MERGED1997_98_PP.csv


C:\Users\Rob.DESKTOP-HBG5EOT\.conda\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (1,1537,1540,1542,1606,1608,1614,1615) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


MERGED1998_99_PP.csv
MERGED1999_00_PP.csv


...

MERGED2019_20_PP.csv


C:\Users\Rob.DESKTOP-HBG5EOT\.conda\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,9,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


Most-Recent-Cohorts-All-Data-Elements.csv


C:\Users\Rob.DESKTOP-HBG5EOT\.conda\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,9,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1379,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1412,1413,1427,1428,1431,1432,1433,1517,1532,1533,1534,1535,1536,1537,1538,1539,1540,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1567,1568,1573,1574,1575,1576,1577,1581,1582,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1604,1605,1606,1608,1610,1611,1614,1615,1616,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693

<class 'pandas.core.frame.DataFrame'>
Int64Index: 176720 entries, 0 to 6693
Columns: 2393 entries, UNITID to filename
dtypes: float64(647), int64(6), object(1740)
memory usage: 3.2+ GB


In [86]:
%%time
alldatadf.to_sql('all_data_fields', db) #ERROR: too many columns

OperationalError: (sqlite3.OperationalError) too many columns on all_data_fields
[SQL: 
CREATE TABLE all_data_fields (
	"index" BIGINT, 
	"UNITID" BIGINT, 
	"OPEID" TEXT, 
	"OPEID6" BIGINT, 
	"INSTNM" TEXT, 
	"CITY" TEXT, 
	"STABBR" TEXT, 
	"ZIP" TEXT, 
	"ACCREDAGENCY" TEXT, 
	"INSTURL" TEXT, 
	"NPCURL" TEXT, 
	"SCH_DEG" FLOAT, 
	"HCM2" FLOAT, 
	"MAIN" BIGINT, 
	"NUMBRANCH" BIGINT, 
	"PREDDEG" BIGINT, 
	"HIGHDEG" BIGINT, 
	"CONTROL" FLOAT, 
	"ST_FIPS" FLOAT, 
	"REGION" FLOAT, 
	"LOCALE" FLOAT, 
	"LOCALE2" FLOAT, 
	"LATITUDE" FLOAT, 
	"LONGITUDE" FLOAT, 
	"CCBASIC" FLOAT, 
	"CCUGPROF" FLOAT, 
	"CCSIZSET" FLOAT, 
	"HBCU" FLOAT, 
	"PBI" FLOAT, 
	"ANNHI" FLOAT, 
	"TRIBAL" FLOAT, 
	"AANAPII" FLOAT, 
	"HSI" FLOAT, 
	"NANTI" FLOAT, 
	"MENONLY" FLOAT, 
	"WOMENONLY" FLOAT, 
	"RELAFFIL" FLOAT, 
	"ADM_RATE" FLOAT, 
	"ADM_RATE_ALL" FLOAT, 
	"SATVR25" FLOAT, 
	"SATVR75" FLOAT, 
	"SATMT25" FLOAT, 
	"SATMT75" FLOAT, 
	"SATWR25" FLOAT, 
	"SATWR75" FLOAT, 
	"SATVRMID" FLOAT, 
	"SATMTMID" FLOAT, 
	"SATW

In [87]:
%%time
alldatadf = pd.read_sql('select * from all_data_fields', db)

OperationalError: (sqlite3.OperationalError) no such table: all_data_fields
[SQL: select * from all_data_fields]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
alldatadf = pd.

In [ ]:
import re
FOSrx = re.compile(r'FieldOfStudyData(?P<start>\d+)\_(?P<end>\d+)', re.I)


In [79]:
%%time
#recentf = r'Most-Recent-Cohorts-Field-of-Study.csv'
FOSdf = pd.DataFrame() #pd.read_csv(z.open(recentf))
#FOSdf['filename'] = recentf 
for x in FOS_files:
    print(x)
    df = pd.read_csv(z.open(x))
    df['filename'] = x
    FOSdf = FOSdf.append(df)
FOSdf.info()

FieldOfStudyData1415_1516_PP.csv
FieldOfStudyData1516_1617_PP.csv
FieldOfStudyData1617_1718_PP.csv
Most-Recent-Cohorts-Field-of-Study.csv
<class 'pandas.core.frame.DataFrame'>
Int64Index: 954880 entries, 0 to 260530
Data columns (total 93 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   UNITID                         930096 non-null  float64
 1   OPEID6                         954880 non-null  int64  
 2   INSTNM                         954880 non-null  object 
 3   CONTROL                        954880 non-null  object 
 4   MAIN                           954880 non-null  int64  
 5   CIPCODE                        954880 non-null  int64  
 6   CIPDESC                        954880 non-null  object 
 7   CREDLEV                        954880 non-null  int64  
 8   CREDDESC                       954880 non-null  object 
 9   IPEDSCOUNT1                    744392 non-null  float64
 10  IPEDSCOUNT2   

In [80]:
%%time
FOSdf.to_sql('field_of_study', db, index=False, if_exists='replace')

In [81]:
%%time
FOSdf = pd.read_sql('select * from field_of_study', db)

In [82]:
FOSdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 954880 entries, 0 to 954879
Data columns (total 93 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   UNITID                         930096 non-null  float64
 1   OPEID6                         954880 non-null  int64  
 2   INSTNM                         954880 non-null  object 
 3   CONTROL                        954880 non-null  object 
 4   MAIN                           954880 non-null  int64  
 5   CIPCODE                        954880 non-null  int64  
 6   CIPDESC                        954880 non-null  object 
 7   CREDLEV                        954880 non-null  int64  
 8   CREDDESC                       954880 non-null  object 
 9   IPEDSCOUNT1                    744392 non-null  float64
 10  IPEDSCOUNT2                    751224 non-null  float64
 11  DEBT_ALL_STGP_ANY_N            954880 non-null  object 
 12  DEBT_ALL_STGP_ANY_MEAN        

In [47]:
MERGED2017 = pd.read_csv(z.open('MERGED2017_18_PP.csv'))
MERGED2017['year'] = 2017

C:\Users\Rob.DESKTOP-HBG5EOT\.conda\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (1729,1909,1910,1911,1912,1913) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [185]:
#field of study data is earnings debt after graduation by year for 2015-2016, 2016-2017, 2017-2018, and 'latest'(2018-2019?)
FOS2015 = pd.read_csv(z.open('FieldOfStudyData1415_1516_PP.csv'))
FOS2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214917 entries, 0 to 214916
Data columns (total 92 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   UNITID                         210133 non-null  float64
 1   OPEID6                         214917 non-null  int64  
 2   INSTNM                         214917 non-null  object 
 3   CONTROL                        214917 non-null  object 
 4   MAIN                           214917 non-null  int64  
 5   CIPCODE                        214917 non-null  int64  
 6   CIPDESC                        214917 non-null  object 
 7   CREDLEV                        214917 non-null  int64  
 8   CREDDESC                       214917 non-null  object 
 9   IPEDSCOUNT1                    184706 non-null  float64
 10  IPEDSCOUNT2                    184774 non-null  float64
 11  DEBT_ALL_STGP_ANY_N            214917 non-null  object 
 12  DEBT_ALL_STGP_ANY_MEAN        

In [186]:
FOSrecent = pd.read_csv(z.open('Most-Recent-Cohorts-Field-of-Study.csv'))
FOSrecent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 260531 entries, 0 to 260530
Data columns (total 92 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   UNITID                         253808 non-null  float64
 1   OPEID6                         260531 non-null  int64  
 2   INSTNM                         260531 non-null  object 
 3   CONTROL                        260531 non-null  object 
 4   MAIN                           260531 non-null  int64  
 5   CIPCODE                        260531 non-null  int64  
 6   CIPDESC                        260531 non-null  object 
 7   CREDLEV                        260531 non-null  int64  
 8   CREDDESC                       260531 non-null  object 
 9   IPEDSCOUNT1                    187439 non-null  float64
 10  IPEDSCOUNT2                    189508 non-null  float64
 11  DEBT_ALL_STGP_ANY_N            260531 non-null  object 
 12  DEBT_ALL_STGP_ANY_MEAN        

In [187]:
FOS2015.columns == FOSrecent.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [189]:
ADErecent = pd.read_csv(z.open('Most-Recent-Cohorts-All-Data-Elements.csv'))
ADErecent.info()

C:\Users\Rob.DESKTOP-HBG5EOT\.conda\envs\myenv\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (6,9,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1379,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1412,1413,1427,1428,1431,1432,1433,1517,1532,1533,1534,1535,1536,1537,1538,1539,1540,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1567,1568,1573,1574,1575,1576,1577,1581,1582,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1604,1605,1606,1608,1610,1611,1614,1615,1616,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6694 entries, 0 to 6693
Columns: 2392 entries, UNITID to DOLPROVIDER
dtypes: float64(639), int64(15), object(1738)
memory usage: 122.2+ MB


In [194]:
pd.Series(ADErecent.columns == MERGED2017.columns).value_counts()

True    2392
dtype: int64